## 20 Newsgroups

In [1]:
import matplotlib
import seaborn
import numpy as np
from sklearn import datasets
import tarfile
import re
import os
import pandas as pd

Getting category names, i.e. folder names of 20 newsgroups folder

In [2]:
path = "20_newsgroups"
for root, dirs, files in os.walk(path, topdown=False):
    newsgroups = np.array(dirs)
print newsgroups

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


Reading files from first folder. Just use a for loop to read in from more folders.

In [3]:
path = "20_newsgroups" + "/" + newsgroups[0]
x = []
for root, dirs, files in os.walk(path):
    for name in files:
        f = open(path + "/" + name)
        x.append(f.read())
#print x[1]

Obtain metadatas From, Subject, Date, Organization, Sender from the headers of each file. Obtain Content from the body of each file and place in the original dataframe.

In [4]:
# Creating the dataframe and assigning columns
original = pd.DataFrame()
original = original.assign(From = [])
original = original.assign(Subject = [])
original = original.assign(Date = [])
original = original.assign(Organization = [])
original = original.assign(Sender = [])
original = original.assign(Content = [])
original['From'] = original['From'].astype(str)
original['Subject'] = original['Subject'].astype(str)
original['Date'] = original['Date'].astype(str)
original['Organization'] = original['Organization'].astype(str)
original['Sender'] = original['Sender'].astype(str)
original['Content'] = original['Content'].astype(str)
#print original.dtypes

# Placing the metadatas and content in the respective cells of the dataframe
for i in range(len(x)):
    data = x[i].split("\n\n", 1)
    email_content = ''
    meta = sorted(data[0].split("\n"))
    for k in meta:
        y = np.array(k.split(":", 1))
        if list(original).__contains__(y[0]):
            original.at[i, y[0]] = y[1]
    original.at[i, 'Content'] = data[1]

## Emails
Extract email address from From and Sender column. From and Sender names were not considered as not much info can be gotten. 
Flag whether the document is from an educational institution by checking the domain for .edu
Extra: extract domain, country etc

In [5]:
# Extracting the email addresses from From column
original['Email_from'] = original.From.map(lambda x : re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None else np.NaN)
print original['Email_from'].head()

# Extracting the email addresses from Sender column
original['Email_sender'] = original.Sender.map(lambda x : np.NaN if pd.isnull(x) else re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None else np.NaN)
print original['Email_sender'].head()

# flag as educational organisation if the email address has .edu domain
original.Edu_org = original.Email_from.map(lambda x : 1 if isinstance(x, basestring) and x[len(x) - 3:].find('edu') is not -1 else 0)

0            mathew@mantis.co.uk
1            mathew@mantis.co.uk
2    I3150101@dbstu1.rz.tu-bs.de
3            mathew@mantis.co.uk
4           strom@Watson.Ibm.Com
Name: Email_from, dtype: object
0                         NaN
1                         NaN
2    postnntp@ibr.cs.tu-bs.de
3                         NaN
4                         NaN
Name: Email_sender, dtype: object


## Subject
For Subject, remove Re: (maybe indicate that its a reply message), throw to content

In [13]:
# Process the subject to remove Re:
original['Subject_processed'] = original.Subject.map(lambda x : x[x.find("FAQ:") + 5:] if x.find("FAQ:") is not -1 
                                                     else x[x.find("Re:") + 4:] if x.find("Re:") is not -1 and x.find("Re", 5) is -1
                                                     else x[x.find("Re:") + 4:x.find("Re", 5)] if x.find("Re:") is not -1
                                                     else x)

# Reply_msg = 1 if the document is a reply message, otherwise 0
original['Reply_msg'] = original.Subject.map(lambda x : 1 if x.find("Re:") == 1 else 0)

# Adds the processed subject to Content column
original.Content = original.Content + " " + original.Subject_processed

# Removes Subject_processed column as they are appended to content column
del original['Subject_processed']

## Date
For Date, extract month and year (year seems to be all in 1993)

In [8]:
# Extracting the Month from the Date column
month_dict = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
original.Month = original.Date.map(lambda y : ''.join(filter(lambda x: x in month_dict, y.split())))

# Extracting the Year from the Date column
original.Year = original.Date.map(lambda x : x.split()[3] if len(x.split()) is 6 else x.split()[2])
original.Year = original.Year.map(lambda x : x[2:] if len(x) is 4 else x)

Checking the columns of original dataframe

In [14]:
list(original)

['From',
 'Subject',
 'Date',
 'Organization',
 'Sender',
 'Content',
 'Email_from',
 'Email_sender',
 'Reply_msg']

The following shows that some columns have missing values, such as Organisation, Sender, Email_from, and Email_sender

In [16]:
original.count()

From            1000
Subject         1000
Date            1000
Organization     946
Sender           457
Content         1000
Email_from       978
Email_sender     454
Reply_msg       1000
dtype: int64